# 1. 라이브러리 설치 및 기본 설정

In [ ]:
# 1. 라이브러리 설치 (최초 1회만 실행됨)
!pip install opencv-python mediapipe tensorflow numpy

import cv2
import mediapipe as mp
import numpy as np
import PIL.Image
import io
import base64
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import os

# 2. 모델 로드 --- 임시로 더미 데이터 사용 중
model_path = 'model.h5'
if os.path.exists(model_path):
    from tensorflow.keras.models import load_model
    model = load_model(model_path)
    print("모델 로드 성공")
else:
    class MockModel:
        def predict(self, x, verbose=0):
            return np.array([[np.random.rand()]]) # 테스트용 랜덤 값
    model = MockModel()
    print("모델 파일 없음: 테스트 모드로 실행됩니다.")

# 3. MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

print("\n설정 완료! 다음 셀을 실행하세요.")

⚠️ 모델 파일 없음: 테스트 모드로 실행됩니다.

🎉 설정 완료! 다음 셀을 실행하세요.


# 2. 웹캠 통신 함수

In [ ]:
def js_to_image(js_reply):
  """JS 객체를 OpenCV 이미지로 변환"""
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def bbox_to_bytes(bbox_array):
  """OpenCV 이미지를 JS 전송용 byte로 변환"""
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return bbox_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "user"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# 3. 메인 실행 (영상)

In [ ]:
# 스트리밍 시작
video_stream()

label_html = 'Starting Posture Analysis...'
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # 1. 이미지 변환
    img = js_to_image(js_reply["img"])

    # [중요] 점 위치 보정을 위해 좌우 반전 (거울 모드)
    # 이걸 해야 내 왼쪽 팔을 들었을 때 화면의 왼쪽 팔에 점이 찍힙니다.
    img = cv2.flip(img, 1)

    # 2. RGB 변환 및 MediaPipe 분석
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    # 3. 그리기 (이제 원본 img 위에 바로 그립니다)
    if results.pose_landmarks:
        try:
            landmarks = results.pose_landmarks.landmark
            row = [[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks]
            X = np.expand_dims(row, axis=0)

            prediction = model.predict(X, verbose=0)[0][0]

            if prediction > 0.5:
                status = "Good"
                color = (0, 255, 0) # 초록
            else:
                status = "Bad"
                color = (0, 0, 255) # 빨강

            label_html = f"Status: {status} ({prediction*100:.1f}%)"

            # 원본 이미지(img) 위에 랜드마크 그리기
            mp_drawing.draw_landmarks(
                img,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
            )

            # 박스 및 텍스트
            cv2.rectangle(img, (0,0), (200, 60), (245, 117, 16), -1)
            cv2.putText(img, status, (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        except Exception as e:
            pass
    else:
        label_html = 'Pose Not Detected'

    # 4. 출력용 이미지 변환 (투명도 없이 꽉 찬 이미지 전송)
    # 원본 비디오 위에 이 이미지를 덮어씌우므로 위치가 완벽하게 맞게 됩니다.
    img_rgba = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    bbox_bytes = bbox_to_bytes(img_rgba)
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

/tmp/ipython-input-122179483.py:10: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
